# Tutorial 1.3: Introduction to Tracing

![](images/4_Notebook-13-Introduction-to-Tracing.png)

## LLM Observability for GenAI Applications

Welcome to the tracing section! This is where MLflow really shines for GenAI development. You'll learn how to gain complete visibility into your LLM applications.

### What You'll Learn
- What is tracing and why it matters for GenAI
- Understanding the trace data model (traces, spans, hierarchy)
- Automatic tracing with autologging
- Tracing multiple frameworks (OpenAI, LangChain)
- Viewing and analyzing traces in MLflow UI
- Debugging with traces uing MLflow Assistant

### Prerequisites
- Completed Notebooks 1.1 and 1.2
- MLflow UI running
- OpenAI API key configured
- (Or keys for your foundational model provider, e.g., Databricks)

### Estimated Time: 20-25 minutes

---
## Step 1: What is Tracing?

### The Problem

Traditional experimenting tracking or logging isn't enough for LLM applications:

```python
# Traditional logging - hard to debug
print("Calling LLM...")
response = llm.generate(prompt)
print(f"Response: {response}")
# What happened inside? How long did it take? What was sent or received?
```

### The Solution: Distributed Tracing

Tracing captures the **complete execution flow** of your application:

```
Trace: RAG Application
├── Span 1: Embed Query [0ms - 200ms]
│   Input: "What is MLflow for GenAI?"
│   Output: [0.123, 0.456, ...]
│   
├── Span 2: Retrieve Documents [200ms - 350ms]
│   Input: [0.123, 0.456, ...]
│   Output: ["MLflow is...", "The platform..."]
│   
└── Span 3: Generate Response [350ms - 1500ms]
    Input: {query, documents}
    Output: "MLflow is an open source ML and GenAI platform..."
    LLM: gpt-5-mini
    Tokens: 150
```

### Key Benefits

1. **Visibility**: See every step in your LLM workflow
2. **Performance**: Identify bottlenecks and latency issues
3. **Debugging**: Trace errors to their exact source
4. **Cost**: Track token usage per operation
5. **Quality**: Inspect inputs/outputs at each step

---
## Step 2: Understanding the Trace Data Model

### Core Concepts

**Trace**: A complete execution of an operation
- Represents one request or workflow
- Contains one or more spans
- Has a root span

**Span**: A single operation within a trace
- Has a start and end time
- Contains inputs and outputs
- Has metadata (model, tokens,latecy, etc.)
- Can have parent-child relationships

**Span Attributes**: Additional metadata
- Model name
- Token counts
- Temperature
- Custom attributes

### Span Types

MLflow defines standard span types:

- `CHAIN`: A sequence of operations
- `LLM`: Language model call
- `RETRIEVER`: Document retrieval
- `EMBEDDING`: Text embedding
- `TOOL`: Tool/function execution
- `AGENT`: Agent reasoning
- `PARSER`: Output parsing or generic intermediate parsing of an outcome 

### Hierarchy Example

```
TRACE (root)
│
└─ SPAN: Agent Executor (AGENT)
   │
   ├─ SPAN: Planning Step (LLM)
   │  └─ attributes: {model: gpt-5, tokens: 50}
   │
   ├─ SPAN: Tool Execution (TOOL)
   │  └─ attributes: {tool: search, query: "..."}
   │
   └─ SPAN: Final Response (LLM)
      └─ attributes: {model: gpt-5, tokens: 150}
```

---
## Step 3: Environment Setup

In [1]:
import mlflow
from dotenv import load_dotenv
from utils.clnt_utils import is_databricks_ai_gateway_client, get_databricks_ai_gateway_client, get_openai_client, get_ai_gateway_model_names

# Load environment
load_dotenv()

# Configure MLflow
mlflow.set_tracking_uri("http://localhost:5000")

use_databricks_provider = is_databricks_ai_gateway_client()
if use_databricks_provider:
    client = get_databricks_ai_gateway_client()
    model_name = get_ai_gateway_model_names()[0]
else:
    # Initialize OpenAI
    client = get_openai_client()
    model_name = "gpt-5-mini"

print("✅ Environment configured: using", "Databricks" if use_databricks_provider else "OpenAI", "client")
print(f"   MLflow version: {mlflow.__version__}")
print(f"   Tracking URI: {mlflow.get_tracking_uri()}")
print(f"   Using model: {model_name}")

✅ Environment configured: using OpenAI client
   MLflow version: 3.10.0
   Tracking URI: http://localhost:5000
   Using model: gpt-5-mini


---
## Step 4: Your First Trace - Manual Example

Before we use autologging, let's understand what tracing captures by without autologging the information. This way we understand the difference and benefits of using autlogging. To some extent, we saw the benefits of autologgin in the previous notebook on experimental tracking LLM calls. So this is just a quick reminder in case you missed that tutorial.

In [2]:
# Create experiment for tracing examples
mlflow.set_experiment("06-tracing-introduction")

# Without tracing - basic call
print("\n📝 Making LLM call WITHOUT tracing...\n")

prompt = "Explain what distributed tracing is in one sentence."
response = client.chat.completions.create(
    model=model_name,
    messages=[{"role": "user", "content": prompt}],
    temperature=1.0,
    max_completion_tokens=1000
)

print(f"Response: {response.choices[0].message.content}")
print("\n❌ What we DON'T know:")
print("   - Exact timing of the call")
print("   - Detailed request/response structure")
print("   - Easy way to correlate with other operations")
print("   - Visual representation of execution")

# check the trace in the UI
print("\n🔍 View trace in MLflow UI: http://localhost:5000")
print("   Navigate to: Traces tab")


2026/02/24 10:19:22 INFO mlflow.tracking.fluent: Experiment with name '06-tracing-introduction' does not exist. Creating a new experiment.



📝 Making LLM call WITHOUT tracing...

Response: Distributed tracing is a method of instrumenting and recording a request’s execution path across multiple services by propagating trace context and collecting timestamped spans and metadata to visualize and diagnose latency, errors, and performance bottlenecks.

❌ What we DON'T know:
   - Exact timing of the call
   - Detailed request/response structure
   - Easy way to correlate with other operations
   - Visual representation of execution

🔍 View trace in MLflow UI: http://localhost:5000
   Navigate to: Traces tab


---
## Step 5: Automatic Tracing with OpenAI Autologging

Now let's enable tracing with a single line of code!

In [3]:
# Enable OpenAI autologging - THIS IS THE MAGIC LINE!
mlflow.openai.autolog()

print("✅ OpenAI autologging enabled")
print("   All OpenAI API calls will now be automatically traced!")

✅ OpenAI autologging enabled
   All OpenAI API calls will now be automatically traced!


In [4]:
# Make the same call - now it's automatically traced!
print("\n🔍 Making LLM call WITH tracing...\n")

response = client.chat.completions.create(
    model=model_name,
    messages=[{"role": "user", "content": prompt}],
    temperature=1.0,
    max_completion_tokens=1000
)

print(f"Response: {response.choices[0].message.content}")
print("\n✅ What we NOW know:")
print("   ✓ Complete request details (model, messages, parameters)")
print("   ✓ Response content and metadata")
print("   ✓ Token usage (prompt, completion, total)")
print("   ✓ Timing information (latency)")
print("   ✓ All captured automatically!")
print("\n🔗 View trace in MLflow UI: http://localhost:5000")
print("   Navigate to: Traces tab")


🔍 Making LLM call WITH tracing...

Response: Distributed tracing is a method for tracking and visualizing the end-to-end path and timing of a single request as it travels across multiple services or components to diagnose latency, errors, and dependencies.

✅ What we NOW know:
   ✓ Complete request details (model, messages, parameters)
   ✓ Response content and metadata
   ✓ Token usage (prompt, completion, total)
   ✓ Timing information (latency)
   ✓ All captured automatically!

🔗 View trace in MLflow UI: http://localhost:5000
   Navigate to: Traces tab


Trace(trace_id=tr-d839fd86a064e3df386509b9a8c9fe88)

### 🎉 What Just Happened?

With `mlflow.openai.autolog()`, MLflow automatically:

1. **Intercepted** the OpenAI API call
2. **Created** a trace with a unique ID
3. **Captured** all inputs (messages, model, parameters)
4. **Captured** all outputs (response, tokens, timing)
5. **Stored** everything in a structured format
6. **Made it available** in the MLflow UI

**Single line of code change required!**

---
## Step 6: Tracing Multiple Sequential Calls

Let's see how tracing helps with multi-step workflows. That is, multiple sequential call, each creating a trace of its own. This would be typical agentic workflow where an agent might make a series of sequentail calls.

In [5]:
# Simple multi-step workflow
print("\n🔄 Multi-step workflow with automatic tracing...\n")

# Step 1: Generate a topic
print("Step 1: Generating topic...")
topic_response = client.chat.completions.create(
    model=model_name,
    messages=[
        {
        "role": "system",
            "content": "You are a helpful AI expert who explains concepts clearly and concisely."
        },
        {
        "role": "user",
        "content": "Suggest one interesting AI topic for a blog post."
    }],
    temperature=1.0,
    max_completion_tokens=1000
)

# get the topic
topic = topic_response.choices[0].message.content
print(f"  Topic: {topic}")

# Step 2: Generate an outline
print("\nStep 2: Creating outline...")
outline_response = client.chat.completions.create(
    model=model_name,
    messages=[{
        "role": "user",
        "content": f"Create a 3-point outline for a blog post about: {topic}"
    }],
    temperature=1.0,
    max_completion_tokens=500
)
outline = outline_response.choices[0].message.content
print(f"  Outline: {outline[:100]}...")

# Step 3: Write the introduction
print("\nStep 3: Writing introduction...")
intro_response = client.chat.completions.create(
    model=model_name,
    messages=[{
        "role": "user",
        "content": f"Write a 4-sentence introduction paragraph for: {topic}"
    }],
    temperature=1.0,
    max_completion_tokens=2000
)
intro = intro_response.choices[0].message.content
print(f"  Introduction: {intro}")

print("\n✅ All three steps completed!")
print("\n📊 Total tokens used:")
total_tokens = (topic_response.usage.total_tokens +
                outline_response.usage.total_tokens +
                intro_response.usage.total_tokens)
print(f"   {total_tokens} tokens")

print("\n🔍 View in MLflow UI:")
print("   You'll see THREE separate traces, one for each call")
print("   Each trace shows timing, tokens, and complete I/O")


🔄 Multi-step workflow with automatic tracing...

Step 1: Generating topic...
  Topic: Topic: "Generative AI as a Co‑Creator — Practical Workflows for Writers, Designers, and Musicians"

Why it’s interesting:
- Generative models (text, image, audio) are moving from novelty to everyday creative tools. Explaining how to use them productively — not just to generate raw output but to augment human workflows — helps readers get real value while avoiding common pitfalls (garbage outputs, over-reliance, copyright issues).

What to cover (short outline):
1. Hook: a short scenario showing an AI-human collaboration that speeds a real project.
2. Quick primer: what generative models do today (GPT, diffusion, music models) in one paragraph.
3. Practical workflows by discipline:
   - Writers: idea generation → outline → draft → edit with AI prompts.
   - Designers: moodboards → rapid mockups → iterative refinement.
   - Musicians: motif generation → arrangement suggestions → production templates.
4

[Trace(trace_id=tr-b31a07138c5279e8fd96cd9ecd4d4b0e), Trace(trace_id=tr-ebd0de11595472a169064368a1fb6d4c), Trace(trace_id=tr-fee7991e24e578adadbb8e4d914935ef)]

### 💡 Observation

Each OpenAI call creates a **separate trace**. 

(However, in the next tutorial, we'll learn how to group related calls into a **single hierarchical trace** using manual instrumentation with `@mlflow.trace` decorator.)

---
## Step 7: Automatic Tracing with LangChain (Framework Integration)

MLflow supports automatic tracing for many frameworks, over 30+. Let's try a popular one LangChain!

LangChain is a Python/JS framework for building LLM-powered applications by providing reusable building blocks for prompts, models, tools/function-calling, retrieval (RAG), memory/state, evalution, and multi-step chains/agents.

In [ ]:
# Install LangChain if needed
!pip install langchain langchain-openai -q

In [6]:
from langchain_core.prompts import ChatPromptTemplate
from utils.clnt_utils import get_langchain_chat_openai_client, get_databricks_ai_gateway_langchain_client

# check if we are running in a Databricks AI Gateway
databricks_provider = is_databricks_ai_gateway_client()
# Enable LangChain autologging
mlflow.langchain.autolog()

print("✅ LangChain autologging enabled")
print("✅ Using", "Databricks AI Gateway" if databricks_provider else "OpenAI", "as provider")

✅ LangChain autologging enabled
✅ Using OpenAI as provider


In [7]:
# Create a simple LangChain chain
print("\n🔗 Creating and running LangChain chain with tracing...\n")

# Define prompt template
prompt_template = ChatPromptTemplate.from_template(
    "You are a {role}. Answer the following question: {question}"
)

# Create LangChain LLM object
if databricks_provider:
    llm = get_databricks_ai_gateway_langchain_client(model_name, temperature=1.0)
else:
    llm = get_langchain_chat_openai_client(model_name, temperature=1.0)

# Create chain using the prompt template and the LLM. 
# The UNIX like Pipe operator | is used to chain the prompt template and the LLM.
chain = prompt_template | llm

# Run chain
response = chain.invoke({
    "role": "helpful AI assistant",
    "question": "What are the benefits of tracing in GenAI applications?"
})

print(f"Response: {response.content}")
print("\n✅ LangChain execution traced!")
print("\n🔍 In the trace, you'll see:")
print("   - Prompt template construction")
print("   - Variable substitution")
print("   - LLM invocation")
print("   - All as separate spans in a hierarchy!")


🔗 Creating and running LangChain chain with tracing...

Response: Tracing — capturing structured, correlated records of requests, prompts, model responses, intermediate steps, system events and metrics — is extremely valuable in generative-AI systems. Key benefits:

- Faster debugging and root-cause analysis  
  - Correlated traces let you follow a single request from client through prompts, tool calls, model outputs and downstream services, making it far quicker to identify where failures, errors or unexpected outputs occur.

- Reproducibility and auditability  
  - Storing prompts, model version, temperature, system messages, tool call logs and runtime context lets you reproduce a particular response later for investigation, audits, or regulatory purposes.

- Safety and risk detection (hallucinations, bias, policy violations)  
  - Traces let you detect patterns of harmful behavior or hallucinations, link them to trigger prompts or model versions, and create mitigation rules or filt

Trace(trace_id=tr-1244848bf4582de4bcb0f00f4118b8e4)

### 🎯 LangChain Tracing Benefits

With LangChain autologging, MLflow automatically traces:

- **Chain execution**: See how data flows through the chain
- **Prompt construction**: View how templates are filled
- **LLM calls**: Standard OpenAI tracing
- **Retrieval steps**: If using RAG components
- **Tool usage**: If using agents with tools

**All without manual instrumentation!**

---
## Step 8: 🚫 Error Tracing and Debugging 🪲

Tracing is especially valuable when things go wrong. And with MLflow 3.9.0+, 
you can use MLflow Assistant to inpsect, debug, and suggest fixes for you.

We will quickly try this here....

### Debugging with Traces

When your LLM application fails:

1. **Find the trace** in the UI (filter by error status)
2. **Identify the failing span** (marked red)
3. **Inspect inputs** that caused the error
4. **View error details** (message, type, stack trace)
5. **Fix and re-run** with full history preserved
6. **Use MLflow Assistant** to inspect and recommend suggestions

### Some Error Scenarios to Debug with MLflow Assistant

MLflow captures *failed* traces just as faithfully as successful ones — every error, partial result, and stack trace is preserved. Below are three realistic failure patterns that developers commonly hit.

**How to use these traces:**
1. Run each cell below (errors are expected — that's the point!)
2. Open the MLflow UI → **Traces** tab → filter by `Status = Error`
3. Click any failed trace, then click **"Ask MLflow Assistant"** 
4. Ask: *"What caused this error and how do I fix it?"*

| # | Error type | Root cause |
|---|-----------|-----------|
| 1 | API request error | Invalid role in the message list |
| 2 | API request error | Invalid tool/function name (spaces not allowed) |
| 3 | Multi-step pipeline | Application bug at step 3 — steps 1 & 2 succeed, step 3 crashes |

In [8]:
# ── Error Scenario 1: Invalid Role in the Message List ──────────────────────
# Valid OpenAI roles are: "system", "user", "assistant", "tool".
# Passing an unknown role ("robot") triggers a BadRequestError.
# Note: Databricks AI Gateway may surface a slightly different error message.

print("=" * 60)
print("🐛 Error Scenario 1: Invalid Message Role")
print("=" * 60)
print("Calling the API with role='robot' (not a valid OpenAI role)...\n")

try:
    response = client.chat.completions.create(
        model=model_name,
        messages=[
            {"role": "robot", "content": "Classify this review as positive or negative."},
            {"role": "user",  "content": "I love this product!"},
        ],
        max_completion_tokens=50,
    )
except Exception as e:
    print(f"❌ {type(e).__name__}: {str(e)[:200]}")

print("\n✅ Failed trace captured — open MLflow UI to inspect it")
print("   Ask MLflow Assistant: 'Why did this request fail and how do I fix it?'")

🐛 Error Scenario 1: Invalid Message Role
Calling the API with role='robot' (not a valid OpenAI role)...

❌ BadRequestError: Error code: 400 - {'error': {'message': "Invalid value: 'robot'. Supported values are: 'system', 'assistant', 'user', 'function', 'tool', and 'developer'.", 'type': 'invalid_request_error', 'param': '

✅ Failed trace captured — open MLflow UI to inspect it
   Ask MLflow Assistant: 'Why did this request fail and how do I fix it?'


Trace(trace_id=tr-cb67fa5aef1d9e89b55bf664331c6bdf)

In [9]:
# ── Error Scenario 2: Invalid Tool/Function Name ─────────────────────────────
# OpenAI function names must match ^[a-zA-Z0-9_-]{1,64}$.
# Using spaces (e.g. "get weather data") is a common mistake that triggers
# a BadRequestError — captured automatically by mlflow.openai.autolog().

print("=" * 60)
print("🐛 Error Scenario 2: Invalid Tool/Function Name")
print("=" * 60)
print("Calling the API with a function name that contains spaces...\n")

# Common mistake: using a natural-language name with spaces instead of
# snake_case or kebab-case (e.g. 'get_weather' or 'get-weather').
broken_tool = {
    "type": "function",
    "function": {
        "name": "get weather data",   # ← BUG: spaces are not allowed
                                         # Fix: use 'get_weather_data'
        "description": "Retrieve the current weather for a given city.",
        "parameters": {
            "type": "object",
            "properties": {
                "city": {"type": "string", "description": "City name"}
            },
            "required": ["city"],
        },
    },
}

try:
    response = client.chat.completions.create(
        model=model_name,
        messages=[{"role": "user", "content": "What is the weather in Paris?"}],
        tools=[broken_tool],
        max_completion_tokens=100,
    )
except Exception as e:
    print(f"❌ {type(e).__name__}: {str(e)[:200]}")

print("\n✅ Failed trace captured — open MLflow UI to inspect it")
print("   Ask MLflow Assistant: 'What is wrong with this tool definition?'")

🐛 Error Scenario 2: Invalid Tool/Function Name
Calling the API with a function name that contains spaces...

❌ BadRequestError: Error code: 400 - {'error': {'message': "Invalid 'tools[0].function.name': string does not match pattern. Expected a string that matches the pattern '^[a-zA-Z0-9_-]+$'.", 'type': 'invalid_request_erro

✅ Failed trace captured — open MLflow UI to inspect it
   Ask MLflow Assistant: 'What is wrong with this tool definition?'


Trace(trace_id=tr-822eb30c32223b6d6e3d583cf2b801cf)

In [10]:
# ── Error Scenario 3: Multi-Step Pipeline — Step 3 Fails ─────────────────────
# A three-step LangChain LCEL pipeline:
#   Step 1 – Generate a product review           (succeeds)
#   Step 2 – Ask the LLM to rate the review      (succeeds, returns "X/10")
#   Step 3 – Parse the rating as a bare integer  (FAILS – "8/10" → ValueError)
#
# mlflow.langchain.autolog() (enabled earlier) records the ENTIRE pipeline as
# ONE hierarchical trace, so you can see each step's output even after the
# pipeline fails — no @mlflow.trace or manual instrumentation needed.

from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnableLambda
from langchain_core.prompts import ChatPromptTemplate

print("=" * 60)
print("🐛 Error Scenario 3: Multi-Step Pipeline — Step 3 Fails")
print("=" * 60)
print("3-step pipeline: steps 1 & 2 succeed, step 3 crashes...\n")



parser = StrOutputParser()
product = "ultra-lightweight noise-cancelling wireless headphones"

def parse_integer_rating(text: str) -> int:
    """BUG: LLM returns 'X/10' format, but int() cannot parse it."""
    return int(text.strip())  # ← raises ValueError when text is e.g. "8/10"

# Build the full LCEL pipeline.
# mlflow.langchain.autolog() captures it as a SINGLE hierarchical trace
# with one span per step — you will see the successful spans for steps 1 & 2
# even though step 3 fails.
pipeline = (
    # Step 1: generate a 2-sentence product review (succeeds)
    ChatPromptTemplate.from_template(
        "Write a 2-sentence product review for: {product}"
    )
    | llm
    | parser
    # Step 2: rate the review — explicitly ask for "X/10" format (succeeds)
    | RunnableLambda(lambda review: {"review": review})
    | ChatPromptTemplate.from_template(
        "Rate this product review from 1 to 10. Reply in the format 'X/10':\n\n{review}"
    )
    | llm
    | parser
    # Step 3: parse the rating as a plain integer (FAILS)
    | RunnableLambda(parse_integer_rating)  # ← "8/10" → ValueError
)

try:
    result = pipeline.invoke({"product": product})
    print(f"Rating: {result}")
except Exception as e:
    print(f"❌ Pipeline failed at Step 3 ({type(e).__name__}): {e}")
    print("\n💡 Steps 1 & 2 succeeded — their outputs are preserved in the trace")

print("\n✅ Hierarchical trace captured — open MLflow UI to inspect it")
print("   You will see:")
print("   ✓ Step 1 span: product review generated  (Success)")
print("   ✓ Step 2 span: LLM returned rating text  (Success)")
print("   ✗ Step 3 span: integer parsing failed    (Error)")
print("\n   Ask MLflow Assistant: 'Which step failed and what did each step return?'")

🐛 Error Scenario 3: Multi-Step Pipeline — Step 3 Fails
3-step pipeline: steps 1 & 2 succeed, step 3 crashes...

❌ Pipeline failed at Step 3 (ValueError): invalid literal for int() with base 10: '8/10'

💡 Steps 1 & 2 succeeded — their outputs are preserved in the trace

✅ Hierarchical trace captured — open MLflow UI to inspect it
   You will see:
   ✓ Step 1 span: product review generated  (Success)
   ✓ Step 2 span: LLM returned rating text  (Success)
   ✗ Step 3 span: integer parsing failed    (Error)

   Ask MLflow Assistant: 'Which step failed and what did each step return?'


Trace(trace_id=tr-8d8178e2be629404dd1f511803edb974)

### Using MLflow Assistant to Debug These Traces

After running the cells above, all three failures are visible in the MLflow UI as **Error** traces.

**Step-by-step:**
1. Go to **http://localhost:5000** → **Traces** tab
2. Use the **Status** filter → select **Error**
3. Click on a trace to expand it
4. For hierarchical traces (Scenarios 3 & 4), click the **red span** to see exactly where it failed
5. Click **"Ask Claude"** and try prompts like:
   - *"What caused this error and how do I fix it?"*
   - *"Which span failed and what was its input?"*
   - *"Show me the corrected code"*

> **Key insight:** Even when a multi-step pipeline fails mid-way, MLflow preserves all the
> *successful* upstream spans — so you can see exactly what data each step received and
> produced before the failure occurred.

---
## Step 9: Best Practices for Tracing

Let's review key best practices.

In [11]:
print("""
╔══════════════════════════════════════════════════════════════╗
║           Tracing Best Practices                             ║
╚══════════════════════════════════════════════════════════════╝

✅ DO:

1. Enable autologging early in development
   - Easier to debug from the start
   - Understand performance characteristics early

2. Use tracing in all environments
   - Development: Full tracing for debugging
   - Staging: Trace all requests
   - Production: Sample or trace all (depending on volume)

3. Combine tracing with experiment tracking
   - Traces give you the "how"
   - Experiments give you the "what" and "why"

4. Review traces regularly
   - Look for performance bottlenecks
   - Identify expensive operations
   - Understand failure patterns

5. Use multiple frameworks
   - MLflow supports 30+ integrations
   - Pick the best tool for each task

❌ DON'T:

1. Ignore traces until there's a problem
   - Proactive monitoring prevents issues

2. Log sensitive data in traces
   - PII, credentials, confidential info
   - Use data masking if needed

3. Assume autologging captures everything
   - Custom operations need manual instrumentation
   - We'll cover this in the next notebook!

4. Over-rely on individual traces
   - Look at aggregate patterns
   - Use traces + metrics together

💡 PRO TIPS:

- Set up trace sampling in high-volume production
- Archive old traces to manage storage
- Use MLflow UI to inspect traces and metrics
- Use Claude Code Assistant to inspect, debug, and suggest fixes

""")


╔══════════════════════════════════════════════════════════════╗
║           Tracing Best Practices                             ║
╚══════════════════════════════════════════════════════════════╝

✅ DO:

1. Enable autologging early in development
   - Easier to debug from the start
   - Understand performance characteristics early

2. Use tracing in all environments
   - Development: Full tracing for debugging
   - Staging: Trace all requests
   - Production: Sample or trace all (depending on volume)

3. Combine tracing with experiment tracking
   - Traces give you the "how"
   - Experiments give you the "what" and "why"

4. Review traces regularly
   - Look for performance bottlenecks
   - Identify expensive operations
   - Understand failure patterns

5. Use multiple frameworks
   - MLflow supports 30+ integrations
   - Pick the best tool for each task

❌ DON'T:

1. Ignore traces until there's a problem
   - Proactive monitoring prevents issues

2. Log sensitive data in traces
   - P

---
## Summary

In this notebook, you learned:

1. What tracing is and why it's critical for GenAI
2. The trace data model (traces, spans, hierarchy)
3. Automatic tracing with `mlflow.openai.autolog()`
4. How to view and analyze traces in the MLflow UI
5. Tracing multi-step workflows
6. Framework integration (LangChain example)
7. Error tracing and debugging with MLflow Assistant
8. Best practices for production use

### Key Takeaways

- **One line of code** enables complete observability
- **Automatic capture** of inputs, outputs, and metadata
- **Framework agnostic** - works with OpenAI, LangChain, LlamaIndex, etc.
- **Essential for debugging** complex LLM workflows

### What's Next?

**Notebook 1.4: Manual Tracing and Advanced Observability**

Learn how to:
- Create custom spans with `@mlflow.trace`
- Add custom attributes to spans
- Build hierarchical traces for complex workflows
- Trace RAG pipelines with retrieval and generation steps
- Trace agentic workflows with tool usage
- Debug production issues with advanced techniques